In [ ]:
%matplotlib widget

import ipywidgets as widgets
from ipywidgets import HBox, VBox, jslink, Box, Layout
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import TransferFunction, bode, lsim
from scipy import signal

In [ ]:
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )

# Bode-Diagramm

## System erster Ordnung

\begin{align*}
    \dot{y}(t) + a y(t) & = b u(t)
\end{align*}
mit $a = b = \frac{1}{100}$ und dem Eingangssignal
\begin{align*}
    u(t) & = \sin(2 \pi f_{\mathrm{u}} t), & f_{\mathrm{u}} & = \frac{\omega_{\mathrm{u}}}{2\pi}
\end{align*}

__Definition System__

In [ ]:
f = np.logspace(-2, 5)
sys = TransferFunction([1], [0.01, 1])
w, mag, phase = bode(sys, f)

__Simulation__

In [ ]:
output = widgets.Output()

with output:
    fig = plt.figure(figsize=(10, 5))
    ax3 = plt.subplot(122)
    ax1 = plt.subplot(221)
    ax2 = plt.subplot(223)

plt.subplots_adjust(wspace=0.2, hspace=0.1)
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.subplots_adjust(bottom=0.1, top=0.93, left=0.125, right=0.9)

ax1.grid() 
ax2.grid()
ax3.grid()
[label.set_visible(False) for label in ax1.get_xticklabels()]
ax2.set_xlabel(r"$\omega$")
ax3.set_xlabel(r"$t$")
ax1.set_ylabel(r"$|G(j \omega)|_{\mathrm{dB}}$")
ax2.set_ylabel(r"$\mathrm{arg}\, G(j\omega)/^{\circ}$")
ax3.set_ylim([-1.1, 1.1])

ax1.semilogx(w, mag)
ax2.semilogx(w, phase)
simInputPlot, = ax3.plot([], [], label=r'$u(t)$')
simOutputPlot, = ax3.plot([], [], label=r'$y(t)$')

magLine = ax1.axvline(x = 0, color = 'r', alpha=0.5)
phaseLine = ax2.axvline(x = 0, color = 'r', alpha=0.5)

handlesAx, labelsAx = ax3.get_legend_handles_labels()
fig.legend([handle for i, handle in enumerate(handlesAx)],
           [label for i, label in enumerate(labelsAx)],
           bbox_to_anchor=(0.5475, 0.94, 0.3525, .15), loc=3,
           ncol=2, mode="expand", borderaxespad=0., framealpha=0.5)

sliderFreq = widgets.FloatLogSlider(value=0.01,
                                    min=-2,
                                    max=4,
                                    step=0.5,
                                    description='Freq')

def calcSystem(_):
    fU = sliderFreq.value
    wU = 2 * np.pi * fU
    
    t = np.linspace(0, 4. / fU, 1001)
    u = np.sin(wU * t)
    tSim, ySim, xSim = lsim(sys, U=u, T=t)

    magLine.set_xdata(np.array([wU]))
    phaseLine.set_xdata(np.array([wU]))
    
    simInputPlot.set_data(tSim, u)
    simOutputPlot.set_data(tSim, ySim)
    ax3.set_xlim([0, t[-1]])
    fig.canvas.draw()

sliderFreq.observe(calcSystem, names='value')

calcSystem(_)

inputControls = Box([sliderFreq])
inputControls.layout = make_box_layout()

HBox([inputControls, output], layout=Layout(display='flex', flex_flow='row', justify_content='center', align_items='center'))